In [1]:
import pandas as pd
import cx_Oracle as cx
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [2]:
class naver_review(): # 네이버 쇼핑에서 리뷰를 수집하는 클래스 객체
    def __init__(self): # 생성자에서 Selenium을 이용한 크롬드라이버 창 실행
        chromedriver_autoinstaller.install()
        self.driver = webdriver.Chrome()
        self.driver.implicitly_wait(2)

    def connect_url(self, url): # 특정 상품 페이지로 접속하는 메서드, url : 접속 대상 url을 텍스트 형식으로 입력
        self.driver.get(url) # url 접속
        time.sleep(1) # 페이지 로드 대기
        self.driver.execute_script("window.scrollTo(0, (document.body.scrollHeight)*0.15);") # 리뷰 메뉴 확인을 위한 스크롤 이동 (전체의 15% 지점)
        time.sleep(0.5) # 페이지 로드 대기
        # 리뷰 메뉴 지정 (해당 CSS 선택자 검색시 총 4개의 요소가 잡히며 상세정보-리뷰-Q&A-반품/교환정보순, 1번째인 리뷰를 선택)
        self.review_btn = self.driver.find_elements(By.CSS_SELECTOR, 'a._11xjFby3Le')[1]
        self.review_btn.click() # 리뷰 메뉴 선택
        print('url : {0} 에 성공적으로 접속했습니다.'.format(url)) # 페이지 접속 성공 메세지 출력
        time.sleep(0.5) # 페이지 로드 대기

    def rip_review(self, x): # 단일 리뷰 페이지의 리뷰 데이터를 수집하는 메서드, x는 크롬드라이버의 .page_source 메서드로 생성된 소스 데이터
        self.soup = BeautifulSoup(x) # 현재 크롬드라이버의 소스 코드를 수집하여 soup 객체 생성
        self.reviews = self.soup.select('div._1McWUwk15j') # 페이지당 20개의 리뷰가 존재하며, 해당 20개의 리뷰 영역을 선택하는 CSS 선택자
        self.review_list = [] # 리뷰 데이터를 담을 리스트 생성

        for i in self.reviews: # 20개의 리뷰 영역에 대하여 반복문 수행
            self.review_temp = {} # 1개 분량의 리뷰 데이터를 담을 딕셔너리 생성

            try: # 선택한 상품 옵션을 추출
                self.review_temp['선택상품'] = i.select('div._2FXNMst_ak')[0].text.strip()
            except:
                self.review_temp['선택상품'] = None

            try: # 리뷰 작성일 추출
                self.review_temp['리뷰작성일'] = i.select('div.iWGqB6S4Lq span._2L3vDiadT9')[0].text.strip()
            except:
                self.review_temp['리뷰작성일'] = None

            try: # 별점을 정수형 데이터로 추출
                self.review_temp['별점'] = int(i.select('em._15NU42F3kT')[0].text.strip())
            except:
                self.review_temp['별점'] = None

            # 추출한 정수형 별점 데이터가 3 이상일 경우 긍정 판정, 별점이 2 이하일 경우 부정, 별점을 추출하지 못했을 경우 결측치로 처리
            if self.review_temp['별점'] >= 3:
                self.review_temp['긍정/부정'] = '긍정'
            elif self.review_temp['별점'] == None:
                self.review_temp['긍정/부정'] = None
            else:
                self.review_temp['긍정/부정'] = '부정'

            try: # 리뷰의 내용을 추출
                self.review_temp['리뷰내용'] = i.select('div._1kMfD5ErZ6 span._2L3vDiadT9')[0].text.replace("\n", " ").strip()
            except:
                self.review_temp['리뷰내용'] = None

            self.review_list.append(self.review_temp) # 추출한 리뷰 정보를 담은 딕셔너리를 리뷰 리스트에 추가
        
        return self.review_list # 최종적으로 20개의 리뷰를 반환
    
    def rip_all(self): # 현재 조회중인 단일 url의 전체 리뷰를 수집하는 메서드
        self.all_reviews = [] # 모든 리뷰를 담을 리스트 생성
        self.idx = 1 # 조회하는 페이지를 판별하는 index 변수 생성

        while True: # 반복을 수행할 횟수가 불분명하므로 무한루프 설정
            self.all_reviews += self.rip_review(self.driver.page_source) # rip_review 메서드를 이용하여 모든 리뷰를 담을 리스트에 데이터를 추가
            self.idx += 1 # 현재 페이지의 리뷰를 모두 수집했으므로 수집할 페이지 변수를 1 증가시킴

            self.button_list = self.driver.find_elements(By.CSS_SELECTOR, 'div._1HJarNZHiI._2UJrM31-Ry a') # 모든 리뷰 페이지 버튼을 탐색하는 CSS 선택자
            self.button_text = [] # 리뷰 페이지 버튼의 실제 텍스트를 담을 임시 리스트 정의
            for i in self.button_list: # 리뷰 페이지 버튼을 담은 리스트에 반복문을 적용하여 버튼의 텍스트를 추출
                try:
                    self.button_text.append(i.text)
                except:
                    self.button_text.append(None)
            
            if self.idx > 200: # 테스트용 리뷰 수집 갯수 제약 (100페이지 x 20개 = 2000개)
                break
                
            try:
            # 탐색하고자 하는 리뷰 페이지가 text의 몇 번째 인덱스인지 확인하여 해당 인덱스 번호를 기존 리뷰 페이지 버튼에 적용하여 다음 페이지 버튼을 지정
                self.btn = self.button_list[self.button_text.index(str(self.idx))] 
                self.btn.click() # 다음 페이지 버튼 클릭
                time.sleep(1) # 페이지 로드 대기
            except:
                try:
            # 탐색하고자 하는 리뷰 페이지가 11, 21 페이지 등에 의하여 다음 페이지로 넘어가야 하는 경우 '다음'으로 되어 있는 버튼을 탐색
            # 기존 idx로는 에러가 발생하므로 예외처리
                    self.btn = self.button_list[self.button_text.index('다음')]
                    self.btn.click() # 다음 페이지 버튼 클릭
                    time.sleep(1) # 페이지 로드 대기
                except:
                    break # 다음 idx 페이지도, '다음' 페이지도 모두 없을 경우 페이지의 끝에 도달한 것으로 판단하여 break 문으로 무한루프 해제 

        return self.all_reviews # 현재까지 수집한 모든 리뷰 데이터를 반환
    
    def page_summary(self): # 현재 조회중인 url에서 리뷰 외 추가적인 상품명, 상품 가격을 추출하는 메서드
        summary = {} # 데이터를 담을 딕셔너리 생성
        summary['product_name'] = self.driver.find_element(By.CSS_SELECTOR, 'h3._22kNQuEXmb').text # 상품명 수집
        summary['product_price'] = int(self.driver.find_element(By.CSS_SELECTOR, 'span._1LY7DqCnwR').text.replace(",", "")) # 상품가격 수집
        summary['reviews'] = self.rip_all() # rip_all로 현재 조회중인 url의 리뷰 데이터 수집

        print("상품 페이지에서 총 {0}개의 리뷰를 수집했습니다.".format(len(summary['reviews']))) # 수집한 데이터의 개수 출력
        return summary # 데이터를 담은 딕셔너리 반환

    def rip_list(self, x): # 여러개의 url에서 데이터를 수집하는 메서드, x : url을 원소로 하는 리스트
        self.all_list = [] # 모든 데이터를 수집할 리스트 정의

        for i in x: # connect_url 메서드로 접속하고, page_summary 메서드로 url의 정보를 수집하는 메서드를 반복문으로 수행하여 리스트에 데이터 수집
            self.connect_url(i)
            self.all_list.append(self.page_summary())

        return self.all_list # 수집한 모든 데이터 반환

    def close(self): # 크롬드라이버를 닫는 메서드
        try:
            self.driver.close()
        except:
            pass

In [3]:
url_list = ['https://smartstore.naver.com/o-ma/products/7363123499',
       'https://brand.naver.com/applestore/products/9360093290',
       'https://smartstore.naver.com/uniyuni/products/6071556962',
       'https://smartstore.naver.com/o-ma/products/7363123499',
       'https://brand.naver.com/sonystore/products/8932776097',
       'https://smartstore.naver.com/cotini/products/5357757813']

In [4]:
test = naver_review()
all_list = test.rip_list(url_list)
test.close()

url : https://smartstore.naver.com/o-ma/products/7363123499 에 성공적으로 접속했습니다.
상품 페이지에서 총 843개의 리뷰를 수집했습니다.
url : https://brand.naver.com/applestore/products/9360093290 에 성공적으로 접속했습니다.
상품 페이지에서 총 1026개의 리뷰를 수집했습니다.
url : https://smartstore.naver.com/uniyuni/products/6071556962 에 성공적으로 접속했습니다.
상품 페이지에서 총 784개의 리뷰를 수집했습니다.
url : https://smartstore.naver.com/o-ma/products/7363123499 에 성공적으로 접속했습니다.
상품 페이지에서 총 843개의 리뷰를 수집했습니다.
url : https://brand.naver.com/sonystore/products/8932776097 에 성공적으로 접속했습니다.
상품 페이지에서 총 1479개의 리뷰를 수집했습니다.
url : https://smartstore.naver.com/cotini/products/5357757813 에 성공적으로 접속했습니다.
상품 페이지에서 총 4000개의 리뷰를 수집했습니다.


In [5]:
all_list

[{'product_name': '갤럭시 버즈2프로 SM-R510',
  'product_price': 137480,
  'reviews': [{'선택상품': '버즈2프로(색상): 화이트',
    '리뷰작성일': '23.12.08.',
    '별점': 5,
    '긍정/부정': '긍정',
    '리뷰내용': '일단 같은 삼성 브랜드 제품과의 호환성이 좋았습니다.  이어팁은 대/중/소 사이즈가 있고 기본은 중 사이즈로, 필요하시면 바꾸시면 됩니다. (워낙 가볍고 작은 제품이라 반드시 안빠지게 처음부터 바꿔줘야함.)  충전은 케이스, 이어폰 모두 상당히 빠르게 이루어지고, 이어폰 완충 시 대략 4시간 정도 사용 가능합니다.  음악 듣는 상태에서 입으로 소리를 내면, 외부 소리가 잘 들리도록 바뀌는 기능이 있고,  소음 차단 설정 시 조용하게 음악을 들을 수 있습니다.  단점  1. 터치에 민감한 편이라 가끔 귀에 닿으면 오작동 할 수 있습니다. 2. 소리는 좋습니다만...     하이엔드 음향브랜드 제품이 더 좋습니다.'},
   {'선택상품': '버즈2프로(색상): 화이트',
    '리뷰작성일': '23.12.09.',
    '별점': 5,
    '긍정/부정': '긍정',
    '리뷰내용': '가성비 대만족합니다. 음질도 좋네요. 타제품 비교 노캔이 약간부족한거같긴한데 그렇게 나쁘지도 않아요. 이제품 나름대로의 노캔 영역이라고 해야할까요? 기본적인 소음은 정말 잘잡는편인것 같아요. 아무튼 총점! 가격까지 비교하면 정말 대만족 ! ㅋ 제가 가진 다른 이어폰들하고 크기비교샷도 찍었습니다.'},
   {'선택상품': '버즈2프로(색상): 화이트',
    '리뷰작성일': '23.11.28.',
    '별점': 5,
    '긍정/부정': '긍정',
    '리뷰내용': '이전에 보스제품 코코에서 구입하고 노캔빼고는 가격대비 별로라 반납하고 버즈 고민하다 구입했는데 제귀가 막귀인지 음질차이 잘모르겠네요. 다음 기종나올때 까지 기다릴까도 생각했지만 나와도

In [6]:
class input_sql(): # CX_ORACLE을 통하여 SQL 데이터에 접근 및 수정하는 클래스 객체 정의
    def __init__(self): # 생성자에서 DB에 접속
        self.dbcon = cx.connect("hr",
                        "hr",
                        "localhost:1521/xe")
        
    def create_table(self): # SQL 테이블을 생성하는 메서드 정의
        self.cursor = self.dbcon.cursor() # SQL문을 입력할 cursor 객체
        # 상품 정보를 담을 포맷을 가진 테이블 생성
        self.sql_base = "create table PRODUCT_LIST (product_id number not null, product varchar2(255) not null, price number not null, url varchar2(255) not null, primary key(product_id))"
        self.sql_base2 = "create sequence SEQ_PRODUCT start with 1 increment by 1"
        self.cursor.execute(self.sql_base)
        self.cursor.execute(self.sql_base2)
        print("테이블 'PRODUCT_LIST'과 시퀀스를 생성했습니다.")
        self.sql = "create table REVIEW_LIST (id number not null, product_id number not null, product_option varchar2(255), reg_date date not null, score number not null, positive_negative varchar2(25) not null, review clob not null, primary key(id), foreign key(product_id) references PRODUCT_LIST(product_id))"
        self.sql2 = "create sequence SEQ_REVIEW start with 1 increment by 1"
        self.cursor.execute(self.sql)
        self.cursor.execute(self.sql2)
        print("테이블 'REVIEW_LIST'과 시퀀스를 생성했습니다.")
        self.cursor.close() # cursor 객체 접속 종료

    def input_data(self, x, y): # 생성한 테이블에 데이터를 삽입하는 메서드 정의
        self.table = x.copy() # naver_review 클래스의 rip_list로 반환받은 리스트를 입력
        self.url = y.copy() # 상품 정보 테이블에 입력할 url 목록을 리스트로 입력
        self.cursor = self.dbcon.cursor() # SQL문을 입력할 cursor 객체
        self.num_data = 0 # 입력한 데이터 개수를 기록할 변수

        # PRODUCT_LIST에 상품 목록을 기입하는 반복문 수행
        for i, v in enumerate(self.table):
            self.sql_base = "insert into PRODUCT_LIST values({0}, '{1}', {2}, '{3}')"
            self.cursor.execute(self.sql_base.format('SEQ_PRODUCT.nextval', v['product_name'], v['product_price'], self.url[i]))

        # REVIEW_LIST에 리뷰 데이터를 기입하는 반복문 수행
        for i, v in enumerate(self.table): # url 개수만큼의 요소를 갖는 리스트 반복문 수행
            for j, w in enumerate(v['reviews']): # 단일 url 요소인 딕셔리에서 'reviews'인 key의 리스트를 반복문으로 수행
                # 데이터를 입력할 SQL문 포맷 생성
                self.sql = "insert into REVIEW_LIST values({0}, '{1}', '{2}', TO_DATE('{3}', 'YY.MM.DD.'), {4}, '{5}', '{6}')"
                try: # 리뷰가 1000자 이하일 경우 데이터가 정상적으로 기입됨
                    self.cursor.execute(self.sql.format('SEQ_REVIEW.nextval', i+1, w['선택상품'],
                                                    w['리뷰작성일'], w['별점'], w['긍정/부정'], w['리뷰내용'].replace("'", "'||CHR(039)||'")))
                except: # 리뷰 글자수가 1000자 이상일 경우 1000자 단위로 TO_CLOB을 적용시키는 예외문
                    self.sql2 = "insert into REVIEW_LIST values({0}, '{1}', '{2}', TO_DATE('{3}', 'YY.MM.DD.'), {4}, '{5}', {6})"
                    self.temp_txt = w['리뷰내용'].replace("'", "'||CHR(039)||'")
                    self.n = len(self.temp_txt) // 1000
                    self.li = []

                    for z in range(0, self.n + 1):
                        self.li.append(self.temp_txt[1000*z:1000*(z+1)])

                    self.txt = ""
                    for a, b in enumerate(self.li):
                        self.txt += "TO_CLOB('{0}')".format(b)
                        if a < len(self.li) - 1:
                            self.txt += "||"

                    self.cursor.execute(self.sql2.format('SEQ_REVIEW.nextval', i+1, w['선택상품'],
                                                    w['리뷰작성일'], w['별점'], w['긍정/부정'], self.txt))

                self.num_data += 1 # 데이터를 입력했을 경우 입력한 데이터 수를 1 증가시킴
            
            print("'REVIEW_LIST' 테이블에 {0} 상품에 대한 {1}개의 리뷰 데이터를 입력했습니다.".format(v['product_name'], self.num_data)) # 입력한 데이터 수 출력
            self.dbcon.commit() # COMMINT SQL문으로 입력한 데이터를 실제로 적용
            self.num_data = 0 # 다음 테이블에서 사용할 입력한 데이터 수를 위하여 0으로 초기화
        self.cursor.close() # cursor 객체 접속 종료

    def del_all(self): # 데이터 입력이 잘못되었거나 완전히 새로운 데이터를 입력할 경우 테이블와 Sequence를 삭제하는 메서드 정의
        self.cursor = self.dbcon.cursor() # SQL문을 실행할 cursor 객체

        # 리뷰 리스트 테이블과 Sequence 삭제
        # REVIEW_LIST가 외래 키로 PRODUCT_LIST의 product_id를 참조하고 있으므로 정상적인 삭제 프로세스로 REVIEW_LIST를 먼저 삭제해야함
        try: 
            self.cursor.execute("drop table REVIEW_LIST")
            self.cursor.execute("drop sequence SEQ_REVIEW")
            print("리뷰 테이블과 시퀀스를 삭제했습니다.")
        except:
            pass

        try: # PRODUCT_LIST 테이블과 Sequence 삭제
            self.cursor.execute("drop table PRODUCT_LIST")
            self.cursor.execute("drop sequence SEQ_PRODUCT")
            print("상품 테이블과 시퀀스를 삭제했습니다.")
        except:
            pass

        self.cursor.close() # cursor 객체 접속 종료

    def close(self): # DB에 접속한 dbcon을 종료하는 메서드
        try:
            self.dbcon.close()
        except:
            pass

In [7]:
testing = input_sql()

In [9]:
testing.create_table()

테이블 'PRODUCT_LIST'를 생성했습니다.
테이블 'REVIEW_LIST'를 생성했습니다.


In [10]:
testing.input_data(all_list, url_list)

'REVIEW_LIST' 테이블에 갤럭시 버즈2프로 SM-R510 상품에 대한 843개의 리뷰 데이터를 입력했습니다.
'REVIEW_LIST' 테이블에 Apple 2023 에어팟 프로 2세대 USB-C 충전 케이스 모델 (MTJV3KH/A) 상품에 대한 1026개의 리뷰 데이터를 입력했습니다.
'REVIEW_LIST' 테이블에 삼성전자 갤럭시 버즈2 SM-R177 상품에 대한 784개의 리뷰 데이터를 입력했습니다.
'REVIEW_LIST' 테이블에 갤럭시 버즈2프로 SM-R510 상품에 대한 843개의 리뷰 데이터를 입력했습니다.
'REVIEW_LIST' 테이블에 소니 WF-1000XM5(블랙) 상품에 대한 1479개의 리뷰 데이터를 입력했습니다.
'REVIEW_LIST' 테이블에 T13 블루투스 이어폰 무선 노이즈 캔슬링 국내AS QCY 상품에 대한 4000개의 리뷰 데이터를 입력했습니다.


In [8]:
testing.del_all()

리뷰 테이블과 Sequence를 삭제했습니다.
상품 테이블과 Sequence를 삭제했습니다.


In [11]:
testing.close()